In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

from data_utils import load_images
import torch

In [2]:
train_dataset = load_images("train", percent=1)
test_dataset = load_images("test", percent=1)
validation_dataset = load_images("validation")

In [7]:
from neural_network import get_dataloader_from_image_dataset
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()


train_dataset.images = scaler.fit_transform(train_dataset.images)
validation_dataset.images = scaler.transform(validation_dataset.images)
test_dataset.images = scaler.transform(test_dataset.images)


train_loader = get_dataloader_from_image_dataset(train_dataset, batch_size=64)
test_loader = get_dataloader_from_image_dataset(
    test_dataset, batch_size=64, is_test=True
)
validation_loader = get_dataloader_from_image_dataset(validation_dataset, batch_size=64)

In [8]:
from neural_network import NeuralNetwork

model = NeuralNetwork()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0005)

NUM_EPOCHS = 20
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)

loss_fn = torch.nn.CrossEntropyLoss()

In [9]:
model.train(True)
for i in range(NUM_EPOCHS):
    print(f"Starting epoch {i+1}/{NUM_EPOCHS}")

    for batch_idx, (image_batch, labels_batch) in enumerate(train_loader):
        # print(batch_idx)
        image_batch = image_batch.to(device)
        labels_batch = labels_batch.to(device)
        # print(f"Batch shape: {image_batch.shape}, Labels shape: {labels_batch.shape}")
        predictions = model(image_batch)
        loss = loss_fn(predictions, labels_batch)

        # Backpropagation and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print(f"Epoch {i+1}/{NUM_EPOCHS}, Batch {batch_idx}, Loss: {loss.item()}")

Starting epoch 1/20
Epoch 1/20, Batch 0, Loss: 10.90382194519043
Epoch 1/20, Batch 100, Loss: 5.001683235168457
Starting epoch 2/20
Epoch 2/20, Batch 0, Loss: 3.3871848583221436
Epoch 2/20, Batch 100, Loss: 2.5908570289611816
Starting epoch 3/20
Epoch 3/20, Batch 0, Loss: 2.8822720050811768
Epoch 3/20, Batch 100, Loss: 2.9093856811523438
Starting epoch 4/20
Epoch 4/20, Batch 0, Loss: 2.4357590675354004
Epoch 4/20, Batch 100, Loss: 1.49681556224823
Starting epoch 5/20
Epoch 5/20, Batch 0, Loss: 2.620511054992676
Epoch 5/20, Batch 100, Loss: 2.167526960372925
Starting epoch 6/20
Epoch 6/20, Batch 0, Loss: 2.0522165298461914
Epoch 6/20, Batch 100, Loss: 2.431464195251465
Starting epoch 7/20
Epoch 7/20, Batch 0, Loss: 2.2272026538848877
Epoch 7/20, Batch 100, Loss: 1.550227165222168
Starting epoch 8/20
Epoch 8/20, Batch 0, Loss: 1.4240323305130005
Epoch 8/20, Batch 100, Loss: 1.6388412714004517
Starting epoch 9/20
Epoch 9/20, Batch 0, Loss: 0.9073531627655029
Epoch 9/20, Batch 100, Loss: 1

In [10]:
correct = 0.0
test_loss = 0.0
size = len(validation_loader.dataset)  # numarul total de exemple din setul de validare
model.to(device)
model.eval()
with torch.no_grad():
    for image_batch, labels_batch in validation_loader:
        image_batch = image_batch.to(device)
        labels_batch = labels_batch.to(device)
        pred = model(
            image_batch
        )  # procesam imaginile folosind reteaua antrenata anterior
        test_loss += loss_fn(pred, labels_batch).item()
        correct += (
            (pred.argmax(1) == labels_batch).type(torch.float).sum().item()
        )  # numaram cate exemple sunt corect clasificate
correct /= size
test_loss /= size
print(f"Accuracy: {(100*correct):>0.1f}%, Loss: {test_loss:>8f} \n")

Accuracy: 68.1%, Loss: 0.016112 



In [11]:
correct = 0.0
test_loss = 0.0
size = len(validation_loader.dataset)  # numarul total de exemple din setul de validare
model.to(device)
model.eval()
predictions = []
with torch.no_grad():
    for image_batch, labels_batch in test_loader:
        image_batch = image_batch.to(device)
        pred = model(
            image_batch
        )  # procesam imaginile folosind reteaua antrenata anterior
        predictions.append(pred)
predictions = torch.cat(predictions, dim=0)
predictions = (
    predictions.argmax(dim=1).cpu().numpy()
)  # convertim la numpy pentru a fi usor de folosit
print(predictions)

[1 3 0 ... 3 1 1]


In [12]:
from data_utils import save_predictions_to_csv

ids = test_dataset.ids

save_predictions_to_csv(predictions, ids, "predictions2.csv")

Predictions saved to predictions2.csv
